# Мешок слов

Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

 + кошки (животные)
 + UNIX-утилита cat для вывода содержимого файлов
 + версии операционной системы OS X, названные в честь семейства кошачьих

Задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу использовать косинусное расстояние.

Импортируем необходимые библиотеки.

In [14]:
import numpy
import re
from scipy import spatial

Читаем файл с предложениями (sentences.txt).

In [2]:
file_obj = open('sentences.txt', 'r')
data_list = file_obj.readlines()

In [18]:
data_list

['In comparison to dogs, cats have not undergone major changes during the domestication process.\n',
 'As cat simply catenates streams of bytes, it can be also used to concatenate binary files, where it will just concatenate sequence of bytes.\n',
 'A common interactive use of cat for a single file is to output the content of a file to standard output.\n',
 'Cats can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small animals.\n',
 'In one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.\n',
 'The domesticated cat and its closest wild ancestor are both diploid organisms that possess 38 chromosomes and roughly 20,000 genes.\n',
 'Domestic cats are similar in size to the other members of the genus Felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).\n',
 'However, if the output is piped or redirected, cat is unnecessary.\n',
 'cat with one named file is safer whe

Вводим функции.

In [6]:
def tokenize(sentences):
    '''
    Эта функция пополняет наш словарный запас, просматривая все наши документы (предложения),
    извлекая слова из каждого предложения, удаляя дубликаты с помощью функции set, 
    и возвращает отсортированный список слов.
    '''
    words = []
    for sentence in sentences:
        w = word_extraction(sentence)
        words.extend(w)
        
    words = sorted(list(set(words)))
    return words

In [3]:
def word_extraction(sentence):
    '''
    Данная функция извлекает слова из документа с помощью регулярных выражений, 
    при этом преобразовывает все слова в нижний регистр и исключает стоп-слова.
    '''
    ignore = []
    words = re.sub("[^A-z]", " ", sentence).split()
    cleaned_text = [w.lower() for w in words if w not in ignore]
    return cleaned_text    

In [7]:
def generate_bow(allsentences): 
    '''
    Функция сопоставляет каждому слову из списка всех слов, встречающихся в предложениях, 
    индекс от нуля до (d - 1), где d — число различных слов в предложениях. Далее она создает
    матрицу размера n * d, где n — число предложений, и заполняет ее: элемент с индексом (i, j) 
    в этой матрице равен количеству вхождений j-го слова в i-е предложение. 
    '''
    vocab = tokenize(allsentences)
    
    empty_array = []
    for sentence in allsentences:
        words = word_extraction(sentence)
        bag_vector = numpy.zeros(len(vocab))
        for w in words:
            for i,word in enumerate(vocab):
                if word == w: 
                    bag_vector[i] += 1
        empty_array.append(bag_vector)
    empty_array = numpy.asarray(empty_array)
        
    return empty_array

Применяем функции к датасету.

In [9]:
result = generate_bow(data_list)
print (result)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]]


Получается матрица размера 22 * 254. То есть у нас 22 предложения и 254 различных слова в данных предложениях.

In [8]:
print (result.shape)

(22, 254)


Находим косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine.

In [16]:
distance = spatial.distance.cdist(result[0:1], result[1:], metric='cosine')

In [17]:
print (distance)

[[0.95275444 0.86447381 0.89517152 0.77708871 0.94023857 0.73273876
  0.92587507 0.88427249 0.90550888 0.83281654 0.88047714 0.83964325
  0.87035926 0.87401184 0.94427218 0.84063619 0.9566445  0.94427218
  0.88854436 0.84275727 0.82503645]]


In [648]:
print (sort_distance)

[[0.73273876 0.77708871 0.82503645 0.83281654 0.83964325 0.84063619
  0.84275727 0.86447381 0.87035926 0.87401184 0.88047714 0.88427249
  0.88854436 0.89517152 0.90550888 0.92587507 0.94023857 0.94427218
  0.94427218 0.95275444 0.9566445 ]]


Ближайшие к предложению в самой первой строке по косинусному расстоянию предложения:

In [22]:
data_list[6]

'Domestic cats are similar in size to the other members of the genus Felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).\n'

In [23]:
data_list[4]

'In one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.\n'

Оба найденных предложения близки по смыслу к расположенному в самой первой строке, так как посвящены теме - кошки (животные).